# EXO

### importer les données json dans votre base de données

In [2]:
import pandas as pd
from pymongo import MongoClient
import json
import streamlit as st
from bson.objectid import ObjectId
import re

In [3]:
client = MongoClient("localhost", 4344)
db = client.library_repository
table = db.libraries

In [4]:
# Lecture du json
df = pd.read_json("./repertoire-bibliotheques.json")


In [5]:
# Import des données
if table.count_documents({}) == 0:    
    records = df.to_dict(orient="records")
    table.insert_many(records)
else:
    print("Table déja alimenté")

Table déja alimenté


In [10]:
list(table.find())

[{'_id': ObjectId('5fd34668ec325ac6f88be097'),
  'datasetid': 'repertoire-bibliotheques',
  'recordid': '864841e628b66f6b3770c7770d356afe38e57d47',
  'fields': {'typeinst': 'Bibliothèque médiathèque municipale ou intercommunale',
   'typeinst_code': '1',
   'catalogue_url_ligne': 'http://bondy.opac3d.fr/s/search.php?action=Accueil',
   'nometablissement': 'Bibliothèque Denis Diderot',
   'commune': 'bondy',
   'descoll': 'Collection encyclopédique et multimédia de lecture publique',
   'telephone': '01.48.50.53.40',
   'services_proposes': 'consultation sur place, prêt à domicile, catalogue en ligne',
   'conditionacces': 'Tous publics',
   'accesweb': 'http://bondy.opac3d.fr/s/search.php?action=Accueil',
   'codepostal': '93140',
   'ark_ref': '/06871/0026831',
   'conditionacces_code': 'T',
   'nomrue': '23, rue Roger-Salengro',
   'voltyp': '150 000 documents en consultation',
   'heuresouverture': 'mardi et vendredi : 14h-19h\nmercredi et samedi : 10h-18h\njeudi : 14h-18h',
   'geo

### combien de bibliotheques se trouve a paris

In [15]:
librariesInParis = table.aggregate([
    { "$match": {
        "fields.commune": "Paris"
    }},
    { "$group": {
        "_id": "Compteur", "Nombre de librairies à Paris": { "$sum": 1}
    }}
])

#list(librariesInParis)

print(table.count_documents({"fields.commune": "Paris"}))

615


### Faites un update sur le champs "commune" pour le passer en lower case et refaite le compte des bibliotheques a paris

In [17]:
for doc in table.find(no_cursor_timeout=True):
    pk = ObjectId(str(doc.get("_id")))
    fields = doc.get('fields')
    city = fields.get('commune')
    
    if city:
        city = city.lower()
        table.update_one(
            {"_id": pk}, # MATCH
            {"$set":{"fields.commune": city} # FIELD TO UPDATE
            })


In [6]:
librariesInParis = table.aggregate([
    { "$match": {
        "fields.commune": "paris"
    }},
    { "$group": {
        "_id": "Compteur", "Nombre de librairies à Paris": { "$sum": 1}
    }}
])

list(librariesInParis)

[{'_id': 'Compteur', 'Nombre de librairies à Paris': 615}]

### afficher tout les noms d'établissements situé 95

In [12]:
etablissementIn95 = table.aggregate([
    { "$match": {
        "fields.codepostal": { "$regex":"^95[0-9]{3}$"}
    }},
    {"$group": {"_id": "$fields.nometablissement"}}
])

# OR

etablissementIn95 = table.aggregate([
    { "$match": {
        "fields.codepostal": { "$regex":"^95[0-9]{3}$"}
    }},
    {"$group": {"_id": {"nom": "$fields.nometablissement", "code": "$fields.codepostal"}}}
])

#OR

etablissementIn95 = table.aggregate([
    { "$match": {
        "fields.codepostal": { "$regex":"^95[0-9]{3}$"}
    }},
    {"$project": {"_id": 0, "fields.nometablissement": 1,  "fields.codepostal": 1}}
])


list(etablissementIn95)

[{'fields': {'codepostal': '95800',
   'nometablissement': "Médiathèque de l'Horloge"}},
 {'fields': {'codepostal': '95170',
   'nometablissement': 'Bibliothèque municipale'}},
 {'fields': {'nometablissement': 'Médiathèque municipale',
   'codepostal': '95110'}},
 {'fields': {'nometablissement': "Ecole nationale supérieure d'arts de Paris-Cergy. Centre de documentation",
   'codepostal': '95000'}},
 {'fields': {'nometablissement': "Val-d'Oise", 'codepostal': '95300'}},
 {'fields': {'nometablissement': "Val-d'Oise", 'codepostal': '95011'}},
 {'fields': {'nometablissement': "Réseau des médiathèques intercommunales du Val Parisis, Val d'Oise",
   'codepostal': '95250'}},
 {'fields': {'nometablissement': 'Médiathèque  Elsa-Triolet et Louis-Aragon',
   'codepostal': '95100'}},
 {'fields': {'codepostal': '95610',
   'nometablissement': 'Bibliothèque Albert Camus'}},
 {'fields': {'nometablissement': 'Ecole supérieure des sciences économiques et commerciales. Learning Center',
   'codepostal':

### requeter la bdd pour creer un dictionnaire comportant le nombre de bibliotheque ouvert par jour de la semaine

In [63]:
day_dict = dict({"lundi": 0 , "mardi": 1 , "mercredi": 2, "jeudi": 3, "vendredi": 4, "samedi": 5, "dimanche": 6})

def increment_interval_days(dictionary: dict, input: str):
    if re.search(" (au) ", input, re.IGNORECASE):
        split_string = input.split()
        au_index = split_string.index("au")
        
        start_day = split_string[au_index - 1].lower()
        end_day = split_string[au_index + 1].lower()
        
        isValidDay = False
        
        for i in day_dict:
            if start_day == i:
                isValidDay = True
                
        if isValidDay == False:
            return dictionary
        
        start_day_day_dict_index = day_dict[start_day]
        
        for i in range(start_day_day_dict_index + 1, 6):
            day = list(day_dict.items())[i][0]
    
            if end_day == day:
                break
                
            dictionary[day] +=1
    
    return dictionary

In [67]:
libraryOpenedOnDay = dict({"lundi": 0, "mardi": 0, "mercredi": 0, "jeudi": 0, "vendredi": 0, "samedi": 0, "dimanche": 0})

for doc in table.find(no_cursor_timeout=True):
    fields = doc.get('fields')
    openedHours = str(fields.get('heuresouverture'))

    libraryOpenedOnDay = increment_interval_days(libraryOpenedOnDay, openedHours)
            
    for i in libraryOpenedOnDay:
        if re.search(i, openedHours, re.IGNORECASE):
            libraryOpenedOnDay[i] +=1

print(libraryOpenedOnDay)

{'lundi': 624, 'mardi': 848, 'mercredi': 859, 'jeudi': 771, 'vendredi': 929, 'samedi': 491, 'dimanche': 70}


### certains jours sont bien plus representé que d'autres pourquoi ?

# TP

### le TP consiste a :
  - acquerir de la donnees se mettant a jour assez régulièrement via une api ou du scrapping
    ex: (meteo, bourse, crypto-monaie ect.)
  - stocker cette données dans votre base mongodb rajouter la date precise si ce n'est pas dans vos data
  - mettre en place un outils permetant de visualiser ses data par rapport a la date
    ex:(streamlit, flask ect.)